# Import Packages

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model, load_model, model_from_json
from tensorflow.keras.layers import Dense, Dropout, Activation, Reshape, Conv2D, AveragePooling2D, Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [4]:
os.makedirs('models', exist_ok=True)
os.makedirs('submissions', exist_ok=True)

# Import Data

In [5]:
%%time
# train_data = pd.read_csv('data/MNIST/train.csv
train_data = pd.read_csv('data/MNIST/mnist_train.csv')
test_data = pd.read_csv('data/MNIST/test.csv')

CPU times: user 8.11 s, sys: 601 ms, total: 8.71 s
Wall time: 8.71 s


In [6]:
(train_data.shape, test_data.shape)

((60000, 785), (28000, 784))

In [7]:
X_train = np.array(train_data.iloc[:, 1:])
y_train = np.array(train_data.iloc[:, 0])

In [8]:
X_test = np.array(test_data)

In [9]:
(X_train.shape, y_train.shape, X_test.shape)

((60000, 784), (60000,), (28000, 784))

# Simple Pre-Process Data

In [10]:
# Scale and convert the train images and add channels
X_train = X_train / 255.0
X_train = X_train.reshape((-1, 28, 28, 1))

# Scale and convert the train images and add channels
X_test = X_test / 255.0
X_test = X_test.reshape((-1, 28, 28, 1))

# One Hot Encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)

In [11]:
X_train.shape, y_train.shape, X_test.shape

((60000, 28, 28, 1), (60000, 10), (28000, 28, 28, 1))

# Build NN Model

## Model 1

In [12]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

set_optm(model)

## Model 2

In [13]:
model = Sequential()

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=36, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

set_optm(model)

## Model 3

In [14]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

set_optm(model)

## Model 4

In [15]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

set_optm(model)

## Set Optimizer

In [13]:
def set_optm(model):
    # optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)
    # optimizer = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)  

    optimizer = tf.train.AdamOptimizer(learning_rate=0.0005, beta1=0.9, beta2=0.999, epsilon=1e-08)

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 14, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 28, 14, 32)        4640      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 28, 7, 16)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 7, 64)         9280      
__________

# Fit Model

## With validation

In [17]:
num_epoch = 100

In [18]:
callback_val = [EarlyStopping(monitor='val_loss', patience=10),
                ModelCheckpoint(filepath='models/weights.{epoch:02d}-{val_loss:.2f}_val.h5',
                                monitor='val_loss',
                                save_best_only=True)]

In [19]:
%%time
model.fit(X_train, y_train, epochs=num_epoch, batch_size=256, validation_split=0.33, callbacks=callback_val)

Train on 40199 samples, validate on 19801 samples
Epoch 1/100
40199/40199 [==============================] - 12s 289us/step - loss: 0.7473 - acc: 0.7439 - val_loss: 0.1397 - val_acc: 0.9582
Epoch 2/100
40199/40199 [==============================] - 9s 221us/step - loss: 0.1784 - acc: 0.9489 - val_loss: 0.0892 - val_acc: 0.9731
Epoch 3/100
40199/40199 [==============================] - 9s 223us/step - loss: 0.1232 - acc: 0.9653 - val_loss: 0.0642 - val_acc: 0.9809
Epoch 4/100
40199/40199 [==============================] - 9s 222us/step - loss: 0.0936 - acc: 0.9735 - val_loss: 0.0540 - val_acc: 0.9848
Epoch 5/100
40199/40199 [==============================] - 9s 222us/step - loss: 0.0768 - acc: 0.9787 - val_loss: 0.0535 - val_acc: 0.9859
Epoch 6/100
40199/40199 [==============================] - 9s 222us/step - loss: 0.0659 - acc: 0.9811 - val_loss: 0.0514 - val_acc: 0.9850
Epoch 7/100
40199/40199 [==============================] - 9s 225us/step - loss: 0.0573 - acc: 0.9838 - val_loss: 0

Epoch 24/100
40199/40199 [==============================] - 9s 223us/step - loss: 0.0171 - acc: 0.9948 - val_loss: 0.0365 - val_acc: 0.9916
Epoch 25/100
40199/40199 [==============================] - 9s 223us/step - loss: 0.0163 - acc: 0.9955 - val_loss: 0.0325 - val_acc: 0.9919
Epoch 26/100
40199/40199 [==============================] - 9s 224us/step - loss: 0.0172 - acc: 0.9946 - val_loss: 0.0347 - val_acc: 0.9912
Epoch 27/100
40199/40199 [==============================] - 9s 222us/step - loss: 0.0152 - acc: 0.9952 - val_loss: 0.0355 - val_acc: 0.9915
Epoch 28/100
40199/40199 [==============================] - 9s 222us/step - loss: 0.0135 - acc: 0.9957 - val_loss: 0.0307 - val_acc: 0.9918
Epoch 29/100
40199/40199 [==============================] - 9s 222us/step - loss: 0.0126 - acc: 0.9962 - val_loss: 0.0363 - val_acc: 0.9915
Epoch 30/100
40199/40199 [==============================] - 9s 221us/step - loss: 0.0146 - acc: 0.9952 - val_loss: 0.0345 - val_acc: 0.9919
Epoch 31/100
40199/4

## With full training data

Without validation set, have to set the num epochs manually without early stopping.

In [20]:
num_epoch = 80

In [21]:
model.fit(X_train, y_train, epochs=num_epoch, batch_size=256)

Epoch 1/80
60000/60000 [==============================] - 12s 198us/step - loss: 0.0228 - acc: 0.9939
Epoch 2/80
60000/60000 [==============================] - 12s 192us/step - loss: 0.0196 - acc: 0.9946
Epoch 3/80
60000/60000 [==============================] - 12s 195us/step - loss: 0.0183 - acc: 0.9948
Epoch 4/80
60000/60000 [==============================] - 12s 197us/step - loss: 0.0143 - acc: 0.9957
Epoch 5/80
60000/60000 [==============================] - 12s 193us/step - loss: 0.0157 - acc: 0.9955
Epoch 6/80
60000/60000 [==============================] - 12s 193us/step - loss: 0.0131 - acc: 0.9962
Epoch 7/80
60000/60000 [==============================] - 12s 193us/step - loss: 0.0130 - acc: 0.9962
Epoch 8/80
60000/60000 [==============================] - 12s 193us/step - loss: 0.0125 - acc: 0.9965
Epoch 9/80
60000/60000 [==============================] - 12s 193us/step - loss: 0.0108 - acc: 0.9970
Epoch 10/80
60000/60000 [==============================] - 12s 195us/step - loss: 

# Saving Model

## Save Entire Model

In [22]:
save_model(model, 'models/model_4.h5')

## Save Architecture and Weights Separately

In [23]:
with open('models/model_4_architecture.json', 'w') as f:
    f.write(model.to_json())

In [24]:
model.save_weights('models/model_4_weights.h5')

# Load Model

## Load Entire Model

In [12]:
# model = load_model("models/model_4.h5")

model = load_model("models/weights.28-0.03_val.h5")
set_optm(model)

## Load Architecture and Weights

In [26]:
with open('models/model_4_architecture.json', 'r') as f:
    model = model_from_json(f.read())

In [30]:
model.load_weights('models/model_4_weights.h5')

# Predict on Test Data

In [17]:
%%time
prediction = model.predict(X_test)
prediction = np.argmax(prediction,axis=1)

CPU times: user 4.4 s, sys: 1.04 s, total: 5.43 s
Wall time: 4.34 s


In [18]:
n_samples_test = X_test.shape[0]

In [19]:
data_predict = {"ImageId":range(1, n_samples_test+1), "Label":prediction}
data_predict = pd.DataFrame(data_predict)
data_predict.to_csv("submissions/val.csv", index=False)

Kaggle Score: 

- 0.98514 (20 epochs, 5 epochs)
- 0.99328 (80 epochs, Model 6)
- 0.99142 (14 epochs with validation, Model 6)
- 0.99914 (using full mnist data)